In [1]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd
from scipy import stats

# グラフを描画するライブラリ
from matplotlib import pyplot as plt
import seaborn as sns

sns.set()

# 統計モデルを推定するライブラリ
import statsmodels.api as sm
import statsmodels.tsa.api as tsa
import statsmodels.formula.api as smf
import pmdarima as pm

# 予測
from sktime.forecasting.arima import AutoARIMA

# 予測の評価指標
from sktime.performance_metrics.forecasting import (
    mean_absolute_scaled_error,
    MeanAbsoluteError,
    mean_absolute_percentage_error,
    mean_absolute_error,
)

# グラフの日本語表記
from matplotlib import rcParams

rcParams["font.family"] = "IPAexGothic"
rcParams["font.sans-serif"] = "Meiryo"

In [2]:
# 表示設定
np.set_printoptions(linewidth=60)
pd.set_option("display.width", 80)

from matplotlib.pylab import rcParams

rcParams["figure.figsize"] = 8, 4

In [3]:
# データの読み込み
ts_sales_bj = pd.read_csv(
    "4-3-1-sales-data.csv",  # ファイル名
    index_col="date",  # インデックスとして扱う列名
    parse_dates=True,  # インデックスを「時間軸」として扱う
    dtype="float",  # データの型(浮動小数点)
)

ts_sales_bj.index.freq = "MS"

# 訓練データとテストデータに分割する
train = ts_sales_bj.loc["2010-01":"2018-12"]
test = ts_sales_bj.loc["2019-01":"2019-12"]

In [4]:
# これは間違い
pm.arima.ndiffs(np.log(train["sales"]), test="kpss")

1

In [5]:
# これは間違い
pm.arima.nsdiffs(np.log(train["sales"]), m=12, test="ocsb")

0

In [6]:
# ---------------------------------------------
# 線形回帰モデル（OLS: Ordinary Least Squares）
# ---------------------------------------------
# 売上(sales)を対数変換し、割引(discount)を説明変数として
# 回帰分析を行う。
# ここで対数を取るのは、売上データの分布を正規分布に近づけ、
# 変動率（%変化）を扱いやすくするため。

ols_model = smf.ols("np.log(sales) ~ discount", data=train).fit()

# ---------------------------------------------
# 残差（residuals）の取得
# ---------------------------------------------
# 残差とは、「実際の値」と「モデルの予測値」の差。
# ARIMA/SARIMAXなどの時系列モデルにこの残差を入力すると、
# トレンドや季節性を取り除いた“純粋な変動部分”を扱える。
ols_resid = ols_model.resid

# ---------------------------------------------
# 残差の確認（最初の3行を表示）
# ---------------------------------------------
print(ols_resid.head(3))

date
2010-01-01   -0.025589
2010-02-01   -0.083576
2010-03-01   -0.129386
dtype: float64


In [7]:
# --------------------------------------------------
# 残差に対して単位根検定を行い、必要な差分次数 d を求める
# --------------------------------------------------
# test='kpss'：KPSS検定（帰無仮説＝定常）を使う。
# KPSS の場合、p値が小さいほど「非定常」と判断されるため、
# 差分を1回ずつ取り、定常と判断されるまで繰り返す。
# この結果が ARIMA モデルの d に対応する。

import pmdarima as pm

d = pm.arima.ndiffs(ols_resid, test="kpss")

print(f"推定された差分次数 d：{d}")

推定された差分次数 d：1


In [8]:
# --------------------------------------------------
# 残差に対して季節差分が必要かを検定する
# --------------------------------------------------
# test='ocsb'：OCSB検定（Osborn, Chui, Smith & Birchenhall, 1988）を利用。
#   帰無仮説：季節差分は不要（すでに季節的に定常）
#   対立仮説：季節差分が必要（季節的非定常）
# m=12 は「季節周期＝12（例：月次データ）」を意味する。
# 結果として返される値（通常は 0 または 1）が、
# SARIMA モデルの季節差分次数 D の推定値となる。
import pmdarima as pm

D = pm.arima.nsdiffs(ols_resid, m=12, test="ocsb")
print(f"推定された季節差分次数 D：{D}")

推定された季節差分次数 D：1
